In [24]:
###RNN MNIST分类
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

#hyperparameters
lr=0.001
training_iters=1000
batch_size=128

n_inputs=28
n_steps=28
n_hidden_units=128
n_classes=10

#tf Graph input
x = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.float32,[None,n_classes])

#define weights
weights = {'in':tf.Variable(tf.random_normal([n_inputs,n_hidden_units])),#(28,128)
           'out':tf.Variable(tf.random_normal([n_hidden_units,n_classes]))#(128,10)
          }
biases = {
    'in':tf.Variable(tf.constant(0.1,shape=[n_hidden_units,])),#(128,)
    'out':tf.Variable(tf.constant(0.1,shape=[n_classes,]))#(10,)
}

def RNN(x,weights,biases):
    #hidden layers for input to cell
    x = tf.reshape(x,[-1,n_inputs])#(128batch,28,28)==>(128*28,28)
    x_in = tf.matmul(x,weights['in'])+biases['in']#(128*28,128)
    x_in = tf.reshape(x_in,[-1,n_steps,n_hidden_units])#(128,28,128)
    
    #cell
    def lstm_cell():
        return tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units,forget_bias=1.0,state_is_tuple=True)
    _init_state = lstm_cell.zero_state(batch_size,dtype=tf.float32)
    
    outputs,states = tf.nn.dynamic_rnn(lstm_cell,x_in,initial_state=_init_state,time_major=False)
    
    #hidden layers for outputs
    results = tf.matmul(states[1],weights['out'])+biases['out']
    return results


pred = RNN(x,weights,biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,lables=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    step = 0
    while step*batch_size<training_iters:
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape([batch_size,n_steps,n_inputs])
        sess.run([train_op],feed_dict={x:batch_x,y:batch_y})
        
        if step%20 == 0:
            print(sess.run(accuracy,feed_dict={x:batch_x,y:batch_y}))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


AttributeError: 'function' object has no attribute 'zero_state'